In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
# forex_data = pd.read_csv('/content/drive/MyDrive/BTP Project files/forex1.csv', encoding=encoding)
forex_data = pd.read_csv('/content/drive/MyDrive/NLP BTP/Augmentated data/ChatGPT 3.5 dataset.csv')
forex_data.head()

,true_sentiment,paraphrased_title
0,Positive,Euro set to gain from the ECB's strong determi...
1,Positive,EURCHF's upward trend likely to persist - ING
2,Neutral,Is a surge in EURCHF indicating a break above ...
3,Positive,EURCHF may prolong its advance towards levels ...
4,Positive,EURCHF expected to rise towards 10130 and proj...


In [3]:

import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/NLP BTP/Augmentated data/ChatGPT 3.5 dataset.csv')

# Count the number of occurrences of each sentiment category
sentiment_counts = df['true_sentiment'].value_counts()

# Display the counts
print(sentiment_counts)


true_sentiment
Neutral     815
Positive    767
Negative    709
Name: count, dtype: int64


In [5]:
import pandas as pd

sentiment_counts = df['true_sentiment'].value_counts()

# Display the counts
print(sentiment_counts)


true_sentiment
Neutral     815
Positive    767
Negative    709
Name: count, dtype: int64


In [6]:
df.shape

(2291, 2)

In [7]:
X = df['paraphrased_title'].to_list()
y = df['true_sentiment'].to_list()

In [8]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [9]:
preds = []
preds_proba = []
tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}

for x in X:
    with torch.no_grad():
        input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
        logits = model(**input_sequence).logits
        scores = {
            k: v
            for k, v in zip(
                model.config.id2label.values(),
                scipy.special.softmax(logits.numpy().squeeze()),
            )
        }
        sentimentFinbert = max(scores, key=scores.get)
        probabilityFinbert = max(scores.values())
        preds.append(sentimentFinbert)
        preds_proba.append(probabilityFinbert)


print(preds)
print(preds_proba)
print(y)


['positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'neutral', 'negative', 'neutral', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'neutral', 'positive', 'neutral', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'neutral', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'negative', 'negative', 'neutral', 'negative', 'positive', 'positive', 'negative', 'neutral', 'negative', 'positive', 'neutral', 'negat

In [10]:
preds_capitalized = [' '.join(word.capitalize() for word in sentiment.split()) for sentiment in preds]

print(f'Accuracy-Score: {accuracy_score(y, preds_capitalized)}')


Accuracy-Score: 0.5862068965517241


In [11]:
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error


# Capitalize predictions
preds_capitalized = [' '.join(word.capitalize() for word in sentiment.split()) for sentiment in preds]

# Accuracy score
accuracy = accuracy_score(y, preds_capitalized)
print(f'Accuracy-Score: {accuracy}')

# Classification report
print("Classification Report:")
print(classification_report(y, preds_capitalized))




Accuracy-Score: 0.5862068965517241
Classification Report:
              precision    recall  f1-score   support

    Negative       0.57      0.74      0.64       709
     Neutral       0.61      0.36      0.45       815
    Positive       0.59      0.69      0.63       767

    accuracy                           0.59      2291
   macro avg       0.59      0.59      0.58      2291
weighted avg       0.59      0.59      0.57      2291

